### Linear and Multilinear Functions

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(-1.13 𝐞₁ + 3.37 𝐞₂ + 1.88 𝐞₃ + 0.0669 𝐞₄,
 -1.13 𝐞₁ + 3.37 𝐞₂ + 1.88 𝐞₃ + 0.0669 𝐞₄)

In [3]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(2.89 𝐞₁ + -3.4 𝐞₂ + 0.201 𝐞₃ + 9.87 𝐞₄,
 2.89 𝐞₁ + -3.4 𝐞₂ + 0.201 𝐞₃ + 9.87 𝐞₄)

In [4]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [5]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-1.16 𝐞₁ + 4.72 𝐞₂ + 3.82 𝐞₃ + -3.55 𝐞₄,
 -1.16 𝐞₁ + 4.72 𝐞₂ + 3.82 𝐞₃ + -3.55 𝐞₄)

In [6]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.624 + 0.452 𝐞₁ + 0.21 𝐞₂ + 0.948 𝐞₃ + -0.58 𝐞₄ + -7.17 𝐞₁₂ + -8.26 𝐞₁₃ + 13.4 𝐞₁₄ + 11.2 𝐞₂₃ + -3.0 𝐞₂₄ + 17.4 𝐞₃₄,
 0.624 + 0.452 𝐞₁ + 0.21 𝐞₂ + 0.948 𝐞₃ + -0.58 𝐞₄ + -7.17 𝐞₁₂ + -8.26 𝐞₁₃ + 13.4 𝐞₁₄ + 11.2 𝐞₂₃ + -3.0 𝐞₂₄ + 17.4 𝐞₃₄)

In [7]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(8.45 𝐞₁₂₃ + -30.3 𝐞₁₂₄ + -17.9 𝐞₁₃₄ + 10.3 𝐞₂₃₄,
 8.45 𝐞₁₂₃ + -30.3 𝐞₁₂₄ + -17.9 𝐞₁₃₄ + 10.3 𝐞₂₃₄)

In [8]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.728 + 2.03 𝐞₁ + 2.76 𝐞₂ + 3.09 𝐞₃ + 0.0918 𝐞₄ + 3.85 𝐞₁₂ + -0.176 𝐞₁₃ + 20.6 𝐞₁₄ + 7.31 𝐞₂₃ + 7.7 𝐞₂₄ + 17.5 𝐞₃₄ + 38.5 𝐞₁₂₃ + -99.2 𝐞₁₂₄ + -60.1 𝐞₁₃₄ + 1.04e+02 𝐞₂₃₄ + 91.6 𝐞₁₂₃₄,
 0.728 + 2.03 𝐞₁ + 2.76 𝐞₂ + 3.09 𝐞₃ + 0.0918 𝐞₄ + 3.85 𝐞₁₂ + -0.176 𝐞₁₃ + 20.6 𝐞₁₄ + 7.31 𝐞₂₃ + 7.7 𝐞₂₄ + 17.5 𝐞₃₄ + 38.5 𝐞₁₂₃ + -99.2 𝐞₁₂₄ + -60.1 𝐞₁₃₄ + 1.04e+02 𝐞₂₃₄ + 91.6 𝐞₁₂₃₄)

In [9]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.624 + 1.67 𝐞₁ + 4.37 𝐞₂ + 1.48 𝐞₃ + 1.82 𝐞₄ + -14.4 𝐞₁₂ + 0.136 𝐞₁₃ + -17.8 𝐞₁₄ + 0.0958 𝐞₂₃ + -6.27 𝐞₂₄ + -24.2 𝐞₃₄ + -6.5 𝐞₁₂₃ + 22.2 𝐞₁₂₄ + 30.8 𝐞₁₃₄ + -21.9 𝐞₂₃₄ + 1.67e+02 𝐞₁₂₃₄,
 0.624 + 1.67 𝐞₁ + 4.37 𝐞₂ + 1.48 𝐞₃ + 1.82 𝐞₄ + -14.4 𝐞₁₂ + 0.136 𝐞₁₃ + -17.8 𝐞₁₄ + 0.0958 𝐞₂₃ + -6.27 𝐞₂₄ + -24.2 𝐞₃₄ + -6.5 𝐞₁₂₃ + 22.2 𝐞₁₂₄ + 30.8 𝐞₁₃₄ + -21.9 𝐞₂₃₄ + 1.67e+02 𝐞₁₂₃₄)

In [10]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.624 + 1.67 𝐞₁ + 4.37 𝐞₂ + 14.4 𝐞₁₂, 0.624 + 1.67 𝐞₁ + 4.37 𝐞₂ + 14.4 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.624 + 0.628 𝐞₁ + 0.0946 𝐞₂ + 1.57 𝐞₃ + 0.0348 𝐞₄ + -7.17 𝐞₁₂ + -15.2 𝐞₁₃ + 5.67 𝐞₁₄ + 15.6 𝐞₂₃ + 1.25 𝐞₂₄ + 15.0 𝐞₃₄,
 0.624 + 0.628 𝐞₁ + 0.0946 𝐞₂ + 1.57 𝐞₃ + 0.0348 𝐞₄ + -7.17 𝐞₁₂ + -15.2 𝐞₁₃ + 5.67 𝐞₁₄ + 15.6 𝐞₂₃ + 1.25 𝐞₂₄ + 15.0 𝐞₃₄)

In [13]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.536 + 1.22 𝐞₁ + 3.98 𝐞₂ + 4.1 𝐞₃ + 1.28 𝐞₄ + 20.1 𝐞₁₂ + 40.3 𝐞₁₃ + 11.4 𝐞₁₄ + 5.89 𝐞₂₃ + 5.5 𝐞₂₄ + 19.1 𝐞₃₄ + -31.3 𝐞₁₂₃ + -33.0 𝐞₁₂₄ + -27.6 𝐞₁₃₄ + 90.9 𝐞₂₃₄ + 2.63e+02 𝐞₁₂₃₄,
 0.536 + 1.22 𝐞₁ + 3.98 𝐞₂ + 4.1 𝐞₃ + 1.28 𝐞₄ + 20.1 𝐞₁₂ + 40.3 𝐞₁₃ + 11.4 𝐞₁₄ + 5.89 𝐞₂₃ + 5.5 𝐞₂₄ + 19.1 𝐞₃₄ + -31.3 𝐞₁₂₃ + -33.0 𝐞₁₂₄ + -27.6 𝐞₁₃₄ + 90.9 𝐞₂₃₄ + 2.63e+02 𝐞₁₂₃₄)

In [14]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(4.58 𝐞₁ + -27.3 𝐞₂ + -44.6 𝐞₃ + 2.71 𝐞₄,
 -5.72 𝐞₁ + -4.37 𝐞₂ + -4.71 𝐞₃ + -3.03 𝐞₄,
 4.58 𝐞₁ + -27.3 𝐞₂ + -44.6 𝐞₃ + 2.71 𝐞₄)

In [15]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(46.3 𝐞₁ + -29.6 𝐞₂ + -6.23 𝐞₃ + 37.1 𝐞₄,
 -4.02 𝐞₁ + 3.49 𝐞₂ + 1.72 𝐞₃ + -10.2 𝐞₄,
 46.3 𝐞₁ + -29.6 𝐞₂ + -6.23 𝐞₃ + 37.1 𝐞₄)

In [16]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

3.53 3.53
-47.5 -47.5
-1.47 -1.47


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(4.58 𝐞₁ + -27.3 𝐞₂ + -44.6 𝐞₃ + 2.71 𝐞₄,
 4.58 𝐞₁ + -27.3 𝐞₂ + -44.6 𝐞₃ + 2.71 𝐞₄)

In [18]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, 4.05e+02 𝐞₁₂₃₄, 4.05e+02 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(4.05e+02, 4.05e+02, 4.05e+02)

In [20]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-4.3 𝐞₁₃, -4.3 𝐞₁₃, -4.3 𝐞₁₃)

In [21]:
#1.17a
outermorphism(g, I, alg, h), Ip

(4.3 𝐞₂₃, 1 𝐞₂₃)

In [22]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(4.3 𝐞₁₂, 1 𝐞₁₂)

In [23]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

4.300954730504877

In [24]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.196 + 0.202 𝐞₁ + 0.426 𝐞₂ + 0.99 𝐞₁₂,
 0.196 + 0.202 𝐞₁ + 0.426 𝐞₂ + 0.896 𝐞₃ + 0.419 𝐞₄ + 0.99 𝐞₁₂ + 0.164 𝐞₁₃ + 0.323 𝐞₁₄ + 0.308 𝐞₂₃ + 0.705 𝐞₂₄ + 0.2 𝐞₃₄ + 0.437 𝐞₁₂₃ + 0.076 𝐞₁₂₄ + 0.232 𝐞₁₃₄ + 0.0768 𝐞₂₃₄ + 0.937 𝐞₁₂₃₄)

In [25]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.928 + 0.458 𝐞₂ + 0.638 𝐞₃ + 0.922 𝐞₂₃,
 0.928 + 0.0611 𝐞₁ + 0.458 𝐞₂ + 0.638 𝐞₃ + 0.463 𝐞₄ + 0.09 𝐞₁₂ + 0.396 𝐞₁₃ + 0.323 𝐞₁₄ + 0.922 𝐞₂₃ + 0.0408 𝐞₂₄ + 0.488 𝐞₃₄ + 0.178 𝐞₁₂₃ + 0.32 𝐞₁₂₄ + 0.0179 𝐞₁₃₄ + 0.822 𝐞₂₃₄ + 0.131 𝐞₁₂₃₄)

In [26]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(-4.33e+02 𝐞₁₂₃ + 1.36e+02 𝐞₁₂₄ + -77.9 𝐞₁₃₄ + -25.9 𝐞₂₃₄,
 -4.33e+02 𝐞₁₂₃ + 1.36e+02 𝐞₁₂₄ + -77.9 𝐞₁₃₄ + -25.9 𝐞₂₃₄,
 1.69e+02 + -91.6 𝐞₁₂ + -2.06e+02 𝐞₁₃ + -3.36e+02 𝐞₁₄ + 83.3 𝐞₂₃ + 1.49e+02 𝐞₂₄ + -2.9e+02 𝐞₃₄,
 1.69e+02 + -91.6 𝐞₁₂ + -2.06e+02 𝐞₁₃ + -3.36e+02 𝐞₁₄ + 83.3 𝐞₂₃ + 1.49e+02 𝐞₂₄ + -2.9e+02 𝐞₃₄)

In [27]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-1.18 + -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
 -1.18 + -6.94 𝐞₁ + 4.31 𝐞₂ + 3.79 𝐞₃ + 5.99 𝐞₄ + 6.26 𝐞₁₂ + 5.36 𝐞₁₃ + 6.28 𝐞₁₄ + 0.845 𝐞₂₃ + 1.35 𝐞₂₄ + 3.41 𝐞₃₄ + 5.4 𝐞₁₂₃ + 1.89 𝐞₁₂₄ + -0.382 𝐞₁₃₄ + 1.82 𝐞₂₃₄ + 0.707 𝐞₁₂₃₄)

In [28]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([0.515 𝐞₁ + 10.6 𝐞₂ + 1.15 𝐞₃ + -4.35 𝐞₄,
  2.92 𝐞₁ + 6.2 𝐞₂ + 1.99 𝐞₃ + -1.79 𝐞₄,
  -2.48 𝐞₁ + 4.48 𝐞₂ + 5.47 𝐞₃ + -5.98 𝐞₄,
  6.9 𝐞₁ + 11.7 𝐞₂ + 5.04 𝐞₃ + -3.92 𝐞₄],
 [0.515 𝐞₁ + 10.6 𝐞₂ + 1.15 𝐞₃ + -4.35 𝐞₄,
  2.92 𝐞₁ + 6.2 𝐞₂ + 1.99 𝐞₃ + -1.79 𝐞₄,
  -2.48 𝐞₁ + 4.48 𝐞₂ + 5.47 𝐞₃ + -5.98 𝐞₄,
  6.9 𝐞₁ + 11.7 𝐞₂ + 5.04 𝐞₃ + -3.92 𝐞₄])

In [29]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-1.18, -1.18, -1.18)

In [30]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-94.8 + 35.0 𝐞₁₂ + 49.1 𝐞₁₃ + 21.4 𝐞₁₄ + -21.3 𝐞₂₃ + 39.3 𝐞₂₄ + -30.1 𝐞₃₄ + 13.8 𝐞₁₂₃₄,
 -94.8 + 35.0 𝐞₁₂ + 49.1 𝐞₁₃ + 21.4 𝐞₁₄ + -21.3 𝐞₂₃ + 39.3 𝐞₂₄ + -30.1 𝐞₃₄ + 13.8 𝐞₁₂₃₄)

In [31]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(2106.2241541039175, 2106.224154103919, 2106.224154103918)

In [32]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(4.73e+02 + -72.8 𝐞₁₂ + -2.94e+02 𝐞₁₃ + -6.36e+02 𝐞₁₄ + 92.2 𝐞₂₃ + 4.67e+02 𝐞₂₄ + -3.3e+02 𝐞₃₄,
 4.73e+02 + -72.8 𝐞₁₂ + -2.94e+02 𝐞₁₃ + -6.36e+02 𝐞₁₄ + 92.2 𝐞₂₃ + 4.67e+02 𝐞₂₄ + -3.3e+02 𝐞₃₄)

In [33]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(6.78e+02, 678.2821245258228)

In [34]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(5.27e+02 𝐞₁ + 3.78e+02 𝐞₂ + 6.74e+02 𝐞₃ + -3.94e+02 𝐞₄,
 5.27e+02 𝐞₁ + 3.78e+02 𝐞₂ + 6.74e+02 𝐞₃ + -3.94e+02 𝐞₄,
 0.858 𝐞₁ + 0.6 𝐞₂ + 0.247 𝐞₃ + 0.823 𝐞₄,
 0.858 𝐞₁ + 0.6 𝐞₂ + 0.247 𝐞₃ + 0.823 𝐞₄)

In [35]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-3.64e-12 𝐞₁ + -1.82e-12 𝐞₂ + -1.25e-12 𝐞₃ + 3.41e-13 𝐞₄

In [36]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([-7.15874477+1.87561653j, -7.15874477-1.87561653j,
         4.40706015+0.j        ,  8.7266598 +0.j        ]),
 array([-7.15874477+1.87561653j, -7.15874477-1.87561653j,
         8.7266598 +0.j        ,  4.40706015+0.j        ]))

In [37]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.07041429, 0.07041429, 0.07041429, 0.07041429, 0.07041429,
       0.07041429, 0.07041429, 0.07041429])

In [38]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(1.9 𝐞₁ + 7.84 𝐞₂ + 6.59 𝐞₃ + -5.49 𝐞₄, 1.9 𝐞₁ + 7.84 𝐞₂ + 6.59 𝐞₃ + -5.49 𝐞₄)

In [39]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-94.81741931575226, -94.81741931575226)

In [40]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(168.64047303993516, (168.64047303993482+0j))

In [41]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(167.5366572946449, (167.5366572946469+0j))

In [42]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(2.11e+03 𝐞₁₂₃₄, 2.11e+03 𝐞₁₂₃₄, 2.11e+03 𝐞₁₂₃₄)

In [43]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(1.31e+03 𝐞₁₂₃₄, 1.31e+03 𝐞₁₂₃₄)

In [44]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(4.61 𝐞₁₂, 4.61 𝐞₁₂)

In [45]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-19.85384887680823, -19.85384887680823)

In [46]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-0.627 𝐞₁ + 1.2 𝐞₂, -0.627 𝐞₁ + 2.98 𝐞₂)

In [47]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(-6.61 𝐞₃, -6.61 𝐞₃)

In [48]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(6.61 𝐞₁₂, 6.61 𝐞₁₂)

In [49]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([2, 3], [(0, 1)]),
 array([-7.15874477+1.87561653j, -7.15874477-1.87561653j,
         4.40706015+0.j        ,  8.7266598 +0.j        ]))

In [50]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[4.40706015 4.40706015 4.40706015 4.40706015] [  4.96773108 -21.83694905 -20.72230137  10.75456237]
[8.7266598 8.7266598 8.7266598 8.7266598] [ 5.78399755 17.71277999  6.84679801 10.0946219 ]
[54.76556406 54.76556406 54.76556406 54.76556406 54.76556406 54.76556406] [  52.39759786   18.79331926 -158.02150547   67.7484496    58.49823669
   98.97045434]


In [51]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[ 3.45516726 12.99995497  2.46752569 -0.56538029] [4.40706015 4.40706015 4.40706015 4.40706015]
[20.44428293  8.80658223  6.60410064 -2.30138714] [8.7266598 8.7266598 8.7266598 8.7266598]
[ 157.97794993  136.31595899 -109.17133495  110.13208145   93.18205793
   35.00751451] [54.76556406 54.76556406 54.76556406 54.76556406 54.76556406 54.76556406]


In [52]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, -2.78e-17, 1.0]

In [53]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(23.0 𝐞₁₂₃, 23.0 𝐞₁₂₃), (-4.01 𝐞₄, -4.01 𝐞₄)]

In [54]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[-5.86602834 -5.86602834 -5.86602834] [ 8.55535471 -6.16603075 -9.08145263]
[-0.43004208 -0.43004208 -0.43004208] [  3.48990158  -9.80209963 -18.34020871]
[9.12707601 9.12707601 9.12707601] [ 5.60802888 14.63136938  7.90987447]
[] []
[] []
[] []
[] []
[-4.01477518] [-4.01477518]


In [55]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[ 3.24978145,  3.8636927 ,  1.85662144, -4.88578758],
        [ 3.8636927 ,  4.07832568,  5.38902808,  2.41710232],
        [ 1.85662144,  5.38902808, -4.49710154,  1.7942652 ],
        [-4.88578758,  2.41710232,  1.7942652 , -4.01477518]]),
 1.54 𝐞₁ + 9.08 𝐞₂ + 5.19 𝐞₃ + -5.6 𝐞₄,
 1.54 𝐞₁ + 9.08 𝐞₂ + 5.19 𝐞₃ + -5.6 𝐞₄)

In [56]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(-5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
 -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄)

In [57]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[ 2.22044605e-16,  2.92527079e+00, -1.38293816e+00,
         -1.28145230e+00],
        [-2.92527079e+00,  8.88178420e-16,  1.30635423e+00,
          1.14340798e+00],
        [ 1.38293816e+00, -1.30635423e+00,  0.00000000e+00,
          1.21395865e+00],
        [ 1.28145230e+00, -1.14340798e+00, -1.21395865e+00,
         -4.44089210e-16]]),
 0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄,
 0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄)

In [58]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(-1.06, -1.06, -2.33e-10 𝐞₁₂ + -2.33e-10 𝐞₁₃ + -9.31e-10 𝐞₁₄ + -4.66e-10 𝐞₂₃)

In [59]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.54 𝐞₁ + 9.08 𝐞₂ + 5.19 𝐞₃ + -5.6 𝐞₄, 1.54 𝐞₁ + 9.08 𝐞₂ + 5.19 𝐞₃ + -5.6 𝐞₄)

In [60]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-7.31e+05 𝐞₁ + 7.01e-11 𝐞₂ + -1.23e-11 𝐞₃ + 4.1e-11 𝐞₄,
 -1.32e-12 𝐞₁ + -1.38e+04 𝐞₂ + -3.49e-12 𝐞₃ + -1.03e-12 𝐞₄,
 4.34e-15 𝐞₁ + 6.53e-14 𝐞₂ + -2.58e+02 𝐞₃ + -1.45e-14 𝐞₄,
 -3.33e-16 𝐞₁ + 4.44e-16 𝐞₂ + 3.33e-16 𝐞₃ + -5.93 𝐞₄]

In [61]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [62]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -1.18 + 8.88e-16 𝐞₁₂ + -4.44e-16 𝐞₁₃ + -8.88e-16 𝐞₂₃ + 4.44e-16 𝐞₂₄ + 4.44e-16 𝐞₃₄,
 -1.11e+02 + -8.88e-15 𝐞₁₂ + -1.78e-15 𝐞₁₃ + 3.55e-15 𝐞₂₃ + -7.11e-15 𝐞₂₄ + 1.78e-15 𝐞₃₄ + -3.55e-15 𝐞₁₂₃₄,
 1.63e+02 + 2.84e-14 𝐞₁₂ + 2.49e-14 𝐞₁₃ + 5.68e-14 𝐞₂₃,
 2.31e+03]

In [63]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([-8.1489194 , -6.8996188 ,  9.58847069,  4.27629792]),
 array([-8.1489194 , -6.8996188 ,  4.27629792,  9.58847069]))

In [64]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.429 𝐞₁ + -0.0717 𝐞₂ + -0.485 𝐞₃ + 0.759 𝐞₄,
  -0.212 𝐞₁ + 0.52 𝐞₂ + -0.764 𝐞₃ + -0.319 𝐞₄,
  -0.664 𝐞₁ + 0.426 𝐞₂ + 0.237 𝐞₃ + 0.567 𝐞₄,
  0.575 𝐞₁ + 0.737 𝐞₂ + 0.354 𝐞₃ + -0.0289 𝐞₄],
 [0.429 𝐞₁ + -0.0717 𝐞₂ + -0.485 𝐞₃ + 0.759 𝐞₄,
  0.212 𝐞₁ + -0.52 𝐞₂ + 0.764 𝐞₃ + 0.319 𝐞₄,
  0.664 𝐞₁ + -0.426 𝐞₂ + -0.237 𝐞₃ + -0.567 𝐞₄,
  0.575 𝐞₁ + 0.737 𝐞₂ + 0.354 𝐞₃ + -0.0289 𝐞₄])

In [65]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-8.1489194 -8.1489194 -8.1489194 -8.1489194]
[66.40488734 66.40488734 66.40488734 66.40488734 66.40488734 66.40488734]


In [66]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([0.208 𝐞₁₂ + -0.43 𝐞₁₃ + 0.0239 𝐞₁₄ + 0.307 𝐞₂₃ + -0.372 𝐞₂₄ + 0.734 𝐞₃₄,
  0.575 𝐞₁ + 0.737 𝐞₂ + 0.354 𝐞₃ + -0.0289 𝐞₄,
  -0.664 𝐞₁ + 0.426 𝐞₂ + 0.237 𝐞₃ + 0.567 𝐞₄],
 [-0.208 𝐞₁₂ + 0.43 𝐞₁₃ + -0.0239 𝐞₁₄ + -0.307 𝐞₂₃ + 0.372 𝐞₂₄ + -0.734 𝐞₃₄,
  0.664 𝐞₁ + -0.426 𝐞₂ + -0.237 𝐞₃ + -0.567 𝐞₄,
  0.575 𝐞₁ + 0.737 𝐞₂ + 0.354 𝐞₃ + -0.0289 𝐞₄])

In [67]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.51931359, -0.64979913, -0.21264346,  0.51269605],
        [-0.64979913, -0.12159174,  0.28745463, -0.69307025],
        [-0.21264346,  0.28745463, -0.90593194, -0.22680371],
        [ 0.51269605, -0.69307025, -0.22680371, -0.45316273]]),
 [1.0000000000000002, -5.551115123125783e-17, 0.0, 1.0000000000000004],
 array([-1.,  1., -1., -1.]))

In [68]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.466 𝐞₁ + -1.13 𝐞₂ + -0.421 𝐞₃ + -0.405 𝐞₄,
 -0.466 𝐞₁ + -1.13 𝐞₂ + -0.421 𝐞₃ + -0.405 𝐞₄)

In [69]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([-0.365 𝐞₁ + 0.543 𝐞₂ + -0.756 𝐞₃,
  -0.855 𝐞₁ + 0.478 𝐞₂ + -0.202 𝐞₃,
  -0.72 𝐞₁ + -0.599 𝐞₂ + -0.351 𝐞₃,
  1.0 𝐞₄,
  1 𝐞₁₂₃],
 [6.072521494408008, 7.98199616663114, 0.4344541484338835, 0.0, 0.0])

In [70]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.338, 0.338)

In [71]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄,
 0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄,
 -4.16e-17)

In [72]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(6.33e-10 + -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
 -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
 -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
 -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄)

In [73]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [74]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  -2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
  3.46 𝐞₁₂₃₄],
 [6.66e-16 + -5.85 𝐞₁₂ + 2.77 𝐞₁₃ + 2.56 𝐞₁₄ + -2.61 𝐞₂₃ + -2.29 𝐞₂₄ + -2.43 𝐞₃₄,
  16.6 + 2.44e-15 𝐞₁₂ + 1.11e-15 𝐞₁₃ + 4.22e-15 𝐞₁₄ + 8.88e-16 𝐞₂₃ + 1.11e-15 𝐞₂₄ + -4e-15 𝐞₃₄ + 13.8 𝐞₁₂₃₄,
  1.33e-15 + -8.4 𝐞₁₂ + 7.91 𝐞₁₃ + -9.04 𝐞₁₄ + 8.86 𝐞₂₃ + -9.57 𝐞₂₄ + -20.2 𝐞₃₄,
  12.0])

In [75]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[-2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 4.2 𝐞₁₂ + -3.95 𝐞₁₃ + 4.52 𝐞₁₄ + -4.43 𝐞₂₃ + 4.78 𝐞₂₄ + 10.1 𝐞₃₄]

In [76]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [77]:
x|F, fm(x), x|F|F, fm(fm(x))

(0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄,
 0.358 𝐞₁ + -1.24 𝐞₂ + 1.4 𝐞₃ + 0.113 𝐞₄,
 -5.73 𝐞₁ + 0.915 𝐞₂ + 2.26 𝐞₃ + 0.18 𝐞₄,
 -5.73 𝐞₁ + 0.915 𝐞₂ + 2.26 𝐞₃ + 0.18 𝐞₄)

In [78]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 16.599485922745718, 11.960458975936621],
 [1,
  -6.661338147750939e-16,
  16.599485922745718,
  -1.3322676295501878e-15,
  11.960458975936607],
 array([-15.84462694,  -0.75485898]),
 array([3.98053099, 0.86882621]),
 array([3.05583697e-16+3.98053099j, 3.05583697e-16-3.98053099j,
        2.22044605e-16+0.86882621j, 2.22044605e-16-0.86882621j]),
 [1,
  33.19897184549144,
  299.4638508513066,
  397.0749408012754,
  143.052578915063])

In [79]:
Ck[1], Ck[1] | Ck[2]

(-2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 4.2 𝐞₁₂ + -3.95 𝐞₁₃ + 4.52 𝐞₁₄ + -4.43 𝐞₂₃ + 4.78 𝐞₂₄ + 10.1 𝐞₃₄)

In [80]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(-2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 -2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 5.55e-17 + 5.55e-17 𝐞₁₂ + 2.22e-16 𝐞₁₄ + 1.11e-16 𝐞₂₃ + -2.22e-16 𝐞₂₄ + 1.11e-16 𝐞₃₄ + 3.46 𝐞₁₂₃₄,
 5.55e-17 + -5.55e-17 𝐞₁₂ + -5.55e-17 𝐞₁₃ + -1.67e-16 𝐞₁₄ + -1.39e-16 𝐞₂₃ + 1.11e-16 𝐞₂₄ + -1.11e-16 𝐞₃₄ + 3.46 𝐞₁₂₃₄,
 -16.6 + 6.92 𝐞₁₂₃₄,
 -0.755 + 1.39e-17 𝐞₁₂ + -2.78e-17 𝐞₁₃,
 -15.8 + 2.22e-16 𝐞₁₂ + 2.22e-16 𝐞₁₃)

In [81]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-15.844626939865947, -0.75485898287977], array([-15.84462694,  -0.75485898]))

In [82]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(-0.132 𝐞₁₂ + 0.193 𝐞₁₃ + -0.364 𝐞₁₄ + 0.359 𝐞₂₃ + -0.26 𝐞₂₄ + -0.61 𝐞₃₄,
 -0.132 𝐞₁₂ + 0.193 𝐞₁₃ + -0.364 𝐞₁₄ + 0.359 𝐞₂₃ + -0.26 𝐞₂₄ + -0.61 𝐞₃₄,
 -2.79 𝐞₁₂ + 1.19 𝐞₁₃ + 1.64 𝐞₁₄ + -1.67 𝐞₂₃ + -0.884 𝐞₂₄ + -0.604 𝐞₃₄,
 -2.79 𝐞₁₂ + 1.19 𝐞₁₃ + 1.64 𝐞₁₄ + -1.67 𝐞₂₃ + -0.884 𝐞₂₄ + -0.604 𝐞₃₄)

In [83]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, m+1)]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    linear_eq_matrix = np.array([[np.prod(Lk[i:i+k]) for i in range(m)] for k in range(m)])
    inv_matrix = np.linalg.inv(linear_eq_matrix)
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.dot(inv_matrix, [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [84]:
# From Graded Symmetry Groups: Plane and Simple, 
# instead of solving linear equations, the solutions are given algebraically
# But we need an GA inverse for even algebra
def bivector_split_(F, alg: Algebra):  
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    blades = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        blades.append(even / odd) # FIXME: the inverse usually fails
    return blades

In [85]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-0.334 + -2.19e-13 𝐞₁ + -1.5e-13 𝐞₂ + 9.54e-14 𝐞₃ + 3.45e-13 𝐞₄ + -5.84e-13 𝐞₁₂ + -5.83e-14 𝐞₁₃ + -3.17e-13 𝐞₁₄ + 3.35e-13 𝐞₂₃ + 1.22e-13 𝐞₂₄ + 4.18e-13 𝐞₃₄ + -5.39e-13 𝐞₁₂₃ + -1.15e-13 𝐞₁₂₄ + 1.16e-13 𝐞₁₃₄ + -2.34e-13 𝐞₂₃₄ + -3.58e-13 𝐞₁₂₃₄,
 [array([ 5.77926983e-14, -6.20909426e-13,  7.05773082e-13,  4.08538069e-13,
         -4.64375576e-13,  4.98912805e-12]),
  array([1., 1., 1., 1., 1., 1.]),
  array([5.36968502e-13, 5.36968502e-13, 5.36968502e-13, 5.36968502e-13,
         5.36968502e-13, 5.36968502e-13]),
  array([ 9.29128623, -0.86480971,  0.76082315,  1.31436589, -1.15632374,
          0.10762773])])

In [86]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

7.11e-14 𝐞₁₂ + 8.53e-14 𝐞₁₃ + -1.78e-15 𝐞₁₄ + -6.39e-14 𝐞₂₃ + -8.17e-14 𝐞₂₄ + -8.7e-14 𝐞₃₄

In [87]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.187 𝐞₁ + -0.164 𝐞₂ + -0.269 𝐞₃ + -0.0833 𝐞₄,
 -0.187 𝐞₁ + -0.164 𝐞₂ + -0.269 𝐞₃ + -0.0833 𝐞₄ + 1.1e-17 𝐞₁₂₃ + 1.53e-18 𝐞₁₂₄ + -3.06e-18 𝐞₁₃₄)

In [88]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(-0.132 𝐞₁₂ + 0.193 𝐞₁₃ + -0.364 𝐞₁₄ + 0.359 𝐞₂₃ + -0.26 𝐞₂₄ + -0.61 𝐞₃₄,
 -0.132 𝐞₁₂ + 0.193 𝐞₁₃ + -0.364 𝐞₁₄ + 0.359 𝐞₂₃ + -0.26 𝐞₂₄ + -0.61 𝐞₃₄)

In [89]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [90]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [91]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(1.56 𝐞₁₂ + 1.19 𝐞₁₃ + -2.52 𝐞₁₄ + -0.674 𝐞₂₃ + 5.02 𝐞₂₄ + 0.0842 𝐞₃₄,
 1.56 𝐞₁₂ + 1.19 𝐞₁₃ + -2.52 𝐞₁₄ + -0.674 𝐞₂₃ + 5.02 𝐞₂₄ + 0.0842 𝐞₃₄)

In [92]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(1.56 𝐞₁₂ + 1.19 𝐞₁₃ + -2.52 𝐞₁₄ + -0.674 𝐞₂₃ + 5.02 𝐞₂₄ + 0.0842 𝐞₃₄,
 1.56 𝐞₁₂ + 1.19 𝐞₁₃ + -2.52 𝐞₁₄ + -0.674 𝐞₂₃ + 5.02 𝐞₂₄ + 0.0842 𝐞₃₄)

In [93]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(-0.0996 𝐞₁₂ + 0.146 𝐞₁₃ + -0.274 𝐞₁₄ + 0.271 𝐞₂₃ + -0.196 𝐞₂₄ + -0.46 𝐞₃₄,
 -0.0996 𝐞₁₂ + 0.146 𝐞₁₃ + -0.274 𝐞₁₄ + 0.271 𝐞₂₃ + -0.196 𝐞₂₄ + -0.46 𝐞₃₄)

In [94]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(-1.6 𝐞₁ + 4.14 𝐞₂ + -12.1 𝐞₃ + -2.69 𝐞₄,
 26.6 𝐞₁ + -4.14 𝐞₂ + -16.5 𝐞₃ + -14.7 𝐞₄,
 54.7 𝐞₁ + 64.8 𝐞₂ + 20.6 𝐞₃ + 58.1 𝐞₄,
 1.11e+02 𝐞₁ + 48.3 𝐞₂ + 11.8 𝐞₃ + 34.0 𝐞₄)

In [95]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(1.39 𝐞₁ + 5.58 𝐞₂ + 1.82 𝐞₃ + -1.13 𝐞₄,
 1.39 𝐞₁ + 5.58 𝐞₂ + 1.82 𝐞₃ + -1.13 𝐞₄)

In [96]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(0.207 𝐞₁ + 1.33 𝐞₂ + 1.31 𝐞₃ + 0.721 𝐞₄,
 1.43 𝐞₁ + 2.62 𝐞₂ + 3.62 𝐞₃ + 2.01 𝐞₄ + -4.99e-17 𝐞₁₂₃ + -5.64e-17 𝐞₁₂₄ + -2.2e-16 𝐞₁₃₄ + -1.71e-16 𝐞₂₃₄)

In [97]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(-2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 -2.93 𝐞₁₂ + 1.38 𝐞₁₃ + 1.28 𝐞₁₄ + -1.31 𝐞₂₃ + -1.14 𝐞₂₄ + -1.21 𝐞₃₄,
 -0.115 𝐞₁₂ + 0.168 𝐞₁₃ + -0.316 𝐞₁₄ + 0.312 𝐞₂₃ + -0.226 𝐞₂₄ + -0.53 𝐞₃₄,
 -0.115 𝐞₁₂ + 0.168 𝐞₁₃ + -0.316 𝐞₁₄ + 0.312 𝐞₂₃ + -0.226 𝐞₂₄ + -0.53 𝐞₃₄)

In [98]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([ 220.91437756,  -90.05763905,  -37.98494583, -100.18234096,
        -90.31035812,    5.0207018 ])

In [99]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.76608666, 0.76608666, 0.76608666, 0.76608665, 0.76608666,
        0.76608666])]

In [100]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([0.8752637630319388], array([0.76608665]), [0.7660866609308261])

In [101]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.557 𝐞₁ + -0.41 𝐞₂ + 0.639 𝐞₃ + -0.569 𝐞₄],
 [-0.557 𝐞₁ + -0.41 𝐞₂ + 0.639 𝐞₃ + -0.569 𝐞₄ + -2.08e-17 𝐞₂₃₄],
 [-0.557 𝐞₁ + -0.41 𝐞₂ + 0.639 𝐞₃ + -0.569 𝐞₄ + -6.94e-18 𝐞₁₂₃ + 2.78e-17 𝐞₂₃₄])

In [102]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.221 𝐞₁ + -0.139 𝐞₂ + 0.982 𝐞₃ + 0.421 𝐞₄],
 [0.221 𝐞₁ + -0.139 𝐞₂ + 0.982 𝐞₃ + 0.421 𝐞₄ + 6.94e-18 𝐞₁₂₃ + -2.78e-17 𝐞₂₃₄],
 [0.221 𝐞₁ + -0.139 𝐞₂ + 0.982 𝐞₃ + 0.421 𝐞₄ + 2.08e-17 𝐞₂₃₄])

In [103]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.0999999974781456], [1.0999999974781456])

In [104]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([0.875 + -0.0958 𝐞₁₂ + 0.393 𝐞₁₃ + 0.0621 𝐞₁₄ + 0.179 𝐞₂₃ + 0.144 𝐞₂₄ + -0.473 𝐞₃₄],
 [0.875 + -0.0958 𝐞₁₂ + 0.393 𝐞₁₃ + 0.0621 𝐞₁₄ + 0.179 𝐞₂₃ + 0.144 𝐞₂₄ + -0.473 𝐞₃₄])

In [105]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.8752637630319386

In [106]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.28574726, -3.18574726])

In [107]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(0.207 𝐞₁ + 1.33 𝐞₂ + 1.31 𝐞₃ + 0.721 𝐞₄,
 0.207 𝐞₁ + 1.33 𝐞₂ + 1.31 𝐞₃ + 0.721 𝐞₄ + 2.08e-17 𝐞₁₂₃ + 2.82e-17 𝐞₁₂₄ + 8.67e-19 𝐞₁₃₄ + 1.3e-18 𝐞₂₃₄)

In [108]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.18574726,  0.87526377,  0.87526377,  4.28574726]),
 [-0.785 𝐞₁ + 0.131 𝐞₂ + -0.0633 𝐞₃ + 0.603 𝐞₄,
  0.0248 𝐞₁ + -0.222 𝐞₂ + 0.958 𝐞₃ + 0.181 𝐞₄,
  -0.614 𝐞₁ + -0.3 𝐞₂ + 0.0834 𝐞₃ + -0.725 𝐞₄,
  -0.083 𝐞₁ + 0.918 𝐞₂ + 0.268 𝐞₃ + -0.279 𝐞₄])

In [109]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.14898558, 10.14898558, 10.14898558, 10.14898558]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([18.36762955, 18.36762955, 18.36762955, 18.36762955])]

In [110]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(0.207 𝐞₁ + 1.33 𝐞₂ + 1.31 𝐞₃ + 0.721 𝐞₄,
 0.207 𝐞₁ + 1.33 𝐞₂ + 1.31 𝐞₃ + 0.721 𝐞₄ + 9.38e-18 𝐞₁₂₃ + -6.72e-18 𝐞₁₂₄ + 3.87e-18 𝐞₁₃₄ + -2.1e-17 𝐞₂₃₄)

In [111]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.624 + 0.167 𝐞₁ + 0.256 𝐞₂ + 0.8 𝐞₃ + 0.299 𝐞₄ + 0.952 𝐞₁₂ + 0.429 𝐞₁₃ + 0.788 𝐞₁₄ + 0.557 𝐞₂₃ + 0.909 𝐞₂₄ + 0.848 𝐞₃₄ + 0.0512 𝐞₁₂₃ + 0.413 𝐞₁₂₄ + 0.133 𝐞₁₃₄ + 0.191 𝐞₂₃₄ + 0.412 𝐞₁₂₃₄,
 0.624 + -0.522 𝐞₁ + 5.9 𝐞₂ + 2.57 𝐞₃ + -1.15 𝐞₄ + 1.54e+02 𝐞₁₂ + 48.3 𝐞₁₃ + -44.5 𝐞₁₄ + -4.32 𝐞₂₃ + 1.33e+02 𝐞₂₄ + 42.0 𝐞₃₄ + -14.5 𝐞₁₂₃ + 91.0 𝐞₁₂₄ + 21.6 𝐞₁₃₄ + 7.88 𝐞₂₃₄ + 1.12e+02 𝐞₁₂₃₄)

In [112]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.624 + 0.167 𝐞₁ + 0.256 𝐞₂ + 0.8 𝐞₃ + 0.299 𝐞₄ + 0.952 𝐞₁₂ + 0.429 𝐞₁₃ + 0.788 𝐞₁₄ + 0.557 𝐞₂₃ + 0.909 𝐞₂₄ + 0.848 𝐞₃₄ + 0.0512 𝐞₁₂₃ + 0.413 𝐞₁₂₄ + 0.133 𝐞₁₃₄ + 0.191 𝐞₂₃₄ + 0.412 𝐞₁₂₃₄,
 0.624 + 0.167 𝐞₁ + 0.256 𝐞₂ + 0.8 𝐞₃ + 0.299 𝐞₄ + 0.952 𝐞₁₂ + 0.429 𝐞₁₃ + 0.788 𝐞₁₄ + 0.557 𝐞₂₃ + 0.909 𝐞₂₄ + 0.848 𝐞₃₄ + 0.0512 𝐞₁₂₃ + 0.413 𝐞₁₂₄ + 0.133 𝐞₁₃₄ + 0.191 𝐞₂₃₄ + 0.412 𝐞₁₂₃₄)

In [113]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄ + -8.52e-18 𝐞₁₂₃ + -2.67e-18 𝐞₁₂₄ + 1.23e-19 𝐞₁₃₄ + -3.57e-18 𝐞₂₃₄,
 0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄ + -1.1e-17 𝐞₁₂₃ + 1.13e-18 𝐞₁₂₄ + 5.91e-17 𝐞₁₃₄ + -3.91e-17 𝐞₂₃₄)

In [114]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([2.78e-17 + 0.171 𝐞₁₂ + -0.75 𝐞₁₃ + -0.157 𝐞₁₄ + 0.111 𝐞₂₃ + 0.158 𝐞₂₄ + -0.589 𝐞₃₄,
  5.55e-17 + -0.144 𝐞₁₂ + 0.59 𝐞₁₃ + 0.0932 𝐞₁₄ + 0.269 𝐞₂₃ + 0.215 𝐞₂₄ + -0.71 𝐞₃₄,
  -1.39e-16 + -0.589 𝐞₁₂ + -0.158 𝐞₁₃ + 0.111 𝐞₁₄ + -0.157 𝐞₂₃ + 0.75 𝐞₂₄ + 0.171 𝐞₃₄],
 [0.744 𝐞₁₂ + 0.00197 𝐞₁₃ + -0.33 𝐞₁₄ + -0.0557 𝐞₂₃ + 0.578 𝐞₂₄ + -0.0232 𝐞₃₄,
  -0.144 𝐞₁₂ + 0.59 𝐞₁₃ + 0.0932 𝐞₁₄ + 0.269 𝐞₂₃ + 0.215 𝐞₂₄ + -0.71 𝐞₃₄,
  -0.144 𝐞₁₂ + 0.59 𝐞₁₃ + 0.0932 𝐞₁₄ + 0.269 𝐞₂₃ + 0.215 𝐞₂₄ + -0.71 𝐞₃₄,
  -0.37 𝐞₁₂ + -0.163 𝐞₁₃ + 0.0698 𝐞₁₄ + 0.804 𝐞₂₃ + 0.318 𝐞₂₄ + 0.291 𝐞₃₄])

In [115]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([-0.785 𝐞₁ + 0.131 𝐞₂ + -0.0633 𝐞₃ + 0.603 𝐞₄,
  0.0248 𝐞₁ + -0.222 𝐞₂ + 0.958 𝐞₃ + 0.181 𝐞₄,
  -0.614 𝐞₁ + -0.3 𝐞₂ + 0.0834 𝐞₃ + -0.725 𝐞₄,
  -0.083 𝐞₁ + 0.918 𝐞₂ + 0.268 𝐞₃ + -0.279 𝐞₄],
 [[-0.785 𝐞₁ + 0.131 𝐞₂ + -0.0633 𝐞₃ + 0.603 𝐞₄,
   -0.785 𝐞₁ + 0.131 𝐞₂ + -0.0633 𝐞₃ + 0.603 𝐞₄,
   -0.785 𝐞₁ + 0.131 𝐞₂ + -0.0633 𝐞₃ + 0.603 𝐞₄],
  [-0.352 𝐞₁ + -0.359 𝐞₂ + 0.813 𝐞₃ + -0.295 𝐞₄,
   -0.352 𝐞₁ + -0.359 𝐞₂ + 0.813 𝐞₃ + -0.295 𝐞₄,
   0.0248 𝐞₁ + -0.222 𝐞₂ + 0.958 𝐞₃ + 0.181 𝐞₄],
  [-0.504 𝐞₁ + -0.104 𝐞₂ + -0.514 𝐞₃ + -0.687 𝐞₄,
   -0.504 𝐞₁ + -0.104 𝐞₂ + -0.514 𝐞₃ + -0.687 𝐞₄,
   -0.614 𝐞₁ + -0.3 𝐞₂ + 0.0834 𝐞₃ + -0.725 𝐞₄],
  [-0.083 𝐞₁ + 0.918 𝐞₂ + 0.268 𝐞₃ + -0.279 𝐞₄,
   -0.083 𝐞₁ + 0.918 𝐞₂ + 0.268 𝐞₃ + -0.279 𝐞₄,
   -0.083 𝐞₁ + 0.918 𝐞₂ + 0.268 𝐞₃ + -0.279 𝐞₄]])

In [116]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄ + -8.52e-18 𝐞₁₂₃ + -2.67e-18 𝐞₁₂₄ + 1.23e-19 𝐞₁₃₄ + -3.57e-18 𝐞₂₃₄,
 0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄)

In [117]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.432 + 0.415 𝐞₁₂ + -0.257 𝐞₁₃ + 0.015 𝐞₁₄ + -0.00988 𝐞₂₃ + 0.259 𝐞₂₄ + 0.431 𝐞₃₄ + -0.568 𝐞₁₂₃₄,
 -0.432 + 0.415 𝐞₁₂ + -0.257 𝐞₁₃ + 0.015 𝐞₁₄ + -0.00988 𝐞₂₃ + 0.259 𝐞₂₄ + 0.431 𝐞₃₄ + -0.568 𝐞₁₂₃₄)

In [118]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.646 + -0.116 𝐞₁₂ + 0.17 𝐞₁₃ + -0.319 𝐞₁₄ + 0.316 𝐞₂₃ + -0.228 𝐞₂₄ + -0.536 𝐞₃₄,
 0.646 + -0.116 𝐞₁₂ + 0.17 𝐞₁₃ + -0.319 𝐞₁₄ + 0.316 𝐞₂₃ + -0.228 𝐞₂₄ + -0.536 𝐞₃₄)

In [119]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(0.679 𝐞₁ + 0.63 𝐞₂ + 0.233 𝐞₃ + 0.961 𝐞₄,
 0.679 𝐞₁ + 0.63 𝐞₂ + 0.233 𝐞₃ + 0.961 𝐞₄)

In [120]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄ + -8.52e-18 𝐞₁₂₃ + -2.67e-18 𝐞₁₂₄ + 1.23e-19 𝐞₁₃₄ + -3.57e-18 𝐞₂₃₄,
 0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄,
 0.434 𝐞₁ + 0.341 𝐞₂ + 0.959 𝐞₃ + 0.781 𝐞₄)

In [121]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(-0.653 𝐞₁₂₃ + 0.0565 𝐞₁₂₄ + 0.73 𝐞₁₃₄ + 0.194 𝐞₂₃₄,
 -0.653 𝐞₁₂₃ + 0.0565 𝐞₁₂₄ + 0.73 𝐞₁₃₄ + 0.194 𝐞₂₃₄)

In [122]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(1.63 𝐞₁₂ + 0.825 𝐞₁₃ + -2.04 𝐞₁₄ + -1.9 𝐞₂₃ + 1.37 𝐞₂₄ + -1.69 𝐞₃₄,
 1.63 𝐞₁₂ + 0.825 𝐞₁₃ + -2.04 𝐞₁₄ + -1.9 𝐞₂₃ + 1.37 𝐞₂₄ + -1.69 𝐞₃₄)

In [123]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(0.0689 𝐞₁ + 0.428 𝐞₂ + 1.26 𝐞₃ + 0.349 𝐞₄ + 2.66 𝐞₁₂₃ + 1.41 𝐞₁₂₄ + 2.08 𝐞₁₃₄ + 0.743 𝐞₂₃₄,
 0.0689 𝐞₁ + 0.428 𝐞₂ + 1.26 𝐞₃ + 0.349 𝐞₄ + 2.66 𝐞₁₂₃ + 1.41 𝐞₁₂₄ + 2.08 𝐞₁₃₄ + 0.743 𝐞₂₃₄)

In [124]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-5.12 + 5.55e-17 𝐞₁₄ + -9.71e-17 𝐞₁₅ + 0.373 𝐞₁₂₃₄ + 3.83 𝐞₁₂₃₅ + 1.22 𝐞₁₂₄₅ + 2.49 𝐞₁₃₄₅ + 0.378 𝐞₂₃₄₅

In [125]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.694 𝐞₁ + 0.394 𝐞₂ + 0.943 𝐞₃ + 0.775 𝐞₄ + 0.589 𝐞₅,
 0.694 𝐞₁ + 0.394 𝐞₂ + 0.943 𝐞₃ + 0.775 𝐞₄ + 0.589 𝐞₅)

In [134]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
# with eigenvalue 1 or -1: identity or a reflection

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,)))
sym_vec = sym_vec / norm(sym_vec)
p = 0
if max_diff(ortho(sym_vec), -sym_vec) < 1e-6:
    p += 1

In [127]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    # blade_norm = norm(blade)
    # we can't simply rely on ortho_m:
    # angle = np.arcsin(blade_norm) has no unique solution in [0, pi].
    # (pi - angle) generates the same sine, but differnt cosine (alpha).
    
    alpha = np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t))
    # half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    # rotors.append(blade_exp(half_blade_ortho))
    rotors.append(simple_rotor_sqrt(alpha + blade))
rotor = reduce(alg.gp, rotors)
if p == 1:
    rotor2 = sym_vec * rotor
else:
    rotor2 = rotor
(-1) ** p * rotor2.reverse().sw(x), ortho(x)

(-0.0264 𝐞₁ + 0.808 𝐞₂ + 1.03 𝐞₃ + -0.833 𝐞₄ + -0.272 𝐞₅ + -3.75e-16 𝐞₁₂₃₄₅,
 -0.0264 𝐞₁ + 0.808 𝐞₂ + 1.03 𝐞₃ + -0.833 𝐞₄ + -0.272 𝐞₅)

In [128]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor, lasenby_rotor)

array([0.22478245, 0.22478245, 0.22478245, 0.22478245, 0.22478245,
       0.22478245, 0.22478245, 0.22478245, 0.22478245, 0.22478245,
       0.22478245, 0.22478245, 0.22478245, 0.22478245, 0.22478245,
       0.22478245])

In [129]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
if p == 1:
    l_rotor2 = l_rotor * sym_vec
else:
    l_rotor2 = l_rotor
(-1) ** p * l_rotor2.reverse().sw(x), ortho(x)

(-0.0264 𝐞₁ + 0.808 𝐞₂ + 1.03 𝐞₃ + -0.833 𝐞₄ + -0.272 𝐞₅ + 1.73e-16 𝐞₁₂₃₄₅,
 -0.0264 𝐞₁ + 0.808 𝐞₂ + 1.03 𝐞₃ + -0.833 𝐞₄ + -0.272 𝐞₅)

In [130]:
# Because sym_vec is orthogonal to rotor, we can extract sym_vec from l_rotor:
terms_ratio(l_rotor2.grade(1), sym_vec), terms_ratio(l_rotor | alg.pseudoscalar((1,)), sym_vec)

(array([0.27804662, 0.27804662, 0.27804662, 0.27804662, 0.27804662]),
 array([0.5348228, 0.5348228, 0.5348228, 0.5348228, 0.5348228]))

In [131]:
# From Graded Symmetry Groups: Plane and Simple
def tangent_split(R, alg: Algebra):  
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [R.grade(2*r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    tangents = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        tangents.append(even / odd) # FIXME: the inverse
    return tangents

# tangents = tangent_split(l_rotor, alg)
# rotors2 = []
# for t in tangents:
#     r = 1+t
#     rotors2.append(r/norm(r))
# rotors2, rotors 